In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import functions as fn
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_PLTCM = pd.read_csv('data/pltcm_manipulated_59604_rev1.csv', header=None)
pd.read_csv('data/pltcm_manipulated_59604_rev1.csv', names=fn.columns_PLTCM, header=None)

,EVENT_ID,PROGRAM_ID,1,2,3,4,5,6,INPUT_WIDTH,INPUT_THICKNESS,...,TARGET_WIDTH,THICKNESS_HSM,THICKNESS,TRIM_WIDTH,WIDTH,MATERIAL_ID,PIECE_ID,WEIGHT,LENGTH,INPUT_LENGTH
0,1,3609,0,0,0,0,0,0,1264.0446,2.55459,...,1222.6404,2.55459,0.477354,1224.67,1222.6381,318580,17167961-03000,13.57540,2998.57,543.95600
1,2,3609,0,0,0,0,0,0,1129.5043,2.73491,...,1089.0250,2.73491,0.564271,1091.06,1089.0281,323346,17172941-01000,21.04121,4387.14,875.23380
2,3,3609,0,0,0,0,0,0,1096.1252,2.71956,...,1057.3627,2.71956,0.614289,1059.39,1057.3580,323357,17172961-04000,15.52353,3061.01,672.14825
3,4,3609,0,0,0,0,0,0,1129.7084,2.73506,...,1089.0250,2.73506,0.564197,1091.06,1089.0281,323344,17172921-05000,21.02293,4386.41,875.66516
4,5,3609,0,0,0,0,0,0,1569.3829,2.63399,...,1528.7625,2.63399,0.569115,1529.56,1528.7650,318740,17168341-04000,17.28409,2558.96,536.53850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59599,59600,22800,0,0,0,0,0,0,1261.3135,5.00986,...,1222.3738,5.00986,1.431855,1222.90,1222.3717,1117991,20165941-06000,17.57078,1271.65,371.10983
59600,59601,22800,0,0,0,0,0,0,1261.1356,4.65018,...,1222.3738,4.65018,1.146508,1222.90,1222.3717,1117999,20165961-07000,17.78187,1612.51,404.14432
59601,59602,22800,0,0,0,0,0,0,1263.5488,2.73054,...,1212.2137,2.73054,0.512399,1212.74,1212.2117,1118016,20166001-07000,17.53634,3656.06,685.52203
59602,59603,22800,0,0,0,0,0,0,1280.8510,2.52041,...,1242.1870,2.52041,0.470304,1245.74,1242.1840,1117987,20165941-02000,20.33075,4472.55,840.73800


In [ ]:
#df_PLTCM.iloc[[0,1,2,3,4,67,68,69,70,71,72,73,74,75],[1,8]]

def fun(df, col_key, col_value):
         keys, values = df.sort_values(col_key).values.T
         ukeys, index = np.unique(keys, True)
         arrays = np.split(values, index[1:])
         df2 = pd.DataFrame({col_key:ukeys, col_value:[list(a) for a in arrays]}, )
         return df2

dfG = fun(df_PLTCM[[1,8]], 1, 8)
dfG

,1,8
0,3609.0,"[1264.0446, 1129.5043, 1096.1252, 1129.7084, 1..."
1,3657.0,"[1259.9236, 1287.8539, 1284.3087, 1288.9408, 1..."
2,3681.0,"[1560.9393, 1259.6716, 1258.5481, 1217.0742, 1..."
3,3710.0,"[1569.8542, 1565.6652, 1571.4286, 1395.5007, 1..."
4,3721.0,"[1275.4269, 1261.335, 1576.0254, 1272.8102, 12..."
...,...,...
519,22519.0,"[1351.1455, 1541.3822, 1507.0881, 1305.5166, 1..."
520,22552.0,"[1265.6625, 1665.4911, 1266.5665, 1260.4731, 1..."
521,22652.0,"[1572.6578, 1575.6455, 1574.8002, 1571.4368, 1..."
522,22757.0,"[1549.4031, 1258.1987, 1761.4673, 1390.3076, 1..."


In [ ]:
#df_PLTCM.groupby(1)[8].apply(list)
#list(df_PLTCM.groupby(1)[8].apply(list).values)
#aimbaskets = df_PLTCM.groupby(1)[8].apply(list).reset_index(name=names_columns[8])
aimbaskets = df_PLTCM.groupby(1)[8].apply(list).reset_index()
aimbaskets = aimbaskets[8].to_list()
binningmembers = list(set(df_PLTCM[8].to_list()))
binningmembers.sort()

singlesupportvalues = []
for j in range(len(aimbaskets)):
    a = []
    res = Counter(aimbaskets[j])
    for i in binningmembers:
        a.append(res[i]/len(aimbaskets[j]))
    singlesupportvalues.append(a)

print(len(aimbaskets))
print(len(singlesupportvalues))
print(len(binningmembers))
print(len(singlesupportvalues[1]))

In [9]:
test = np.array(singlesupportvalues)
test2=test.transpose().tolist()
len(test2[1])

524

In [15]:
#list(itertools.product(range(1,3), repeat=2))
#pairs = list(itertools.product(binningmembers, repeat=2))
#pairs